In [1]:
import duckdb
import time
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [2]:
con = duckdb.connect(database="tpch_sf1.db")

In [3]:
#5D version
query_template = """select
	sum(l_extendedprice * l_discount) as revenue
from
	lineitem
where
	l_shipdate >= '$shipdate1'
	and l_shipdate < '$shipdate2'
	and l_discount >= '$discount1' 
    and l_discount < '$discount2'
	and l_quantity < '$quantity';"""


In [4]:
#3D version
query_template = """select
	sum(l_extendedprice * l_discount) as revenue
	from
	lineitem
	where
	l_shipdate >= $shipdate
	and l_discount >= $discount
	and l_quantity < $quantity;
"""

In [5]:
min_shipdate = con.sql("SELECT MIN(l_shipdate) FROM lineitem").fetchone()[0]
max_shipdate = con.sql("SELECT MAX(l_shipdate) FROM lineitem").fetchone()[0]
(min_shipdate, max_shipdate)

(datetime.date(1992, 1, 2), datetime.date(1998, 12, 1))

In [6]:
min_discount = float(con.sql("SELECT MIN(l_discount) FROM lineitem").fetchone()[0])
max_discount = float(con.sql("SELECT MAX(l_discount) FROM lineitem").fetchone()[0])
(min_discount, max_discount)

(0.0, 0.1)

In [7]:
min_quantity = float(con.sql("SELECT MIN(l_quantity) FROM lineitem").fetchone()[0])
max_quantity = float(con.sql("SELECT MAX(l_quantity) FROM lineitem").fetchone()[0])
(min_quantity, max_quantity)

(1.0, 50.0)

In [8]:
table = []
NUM_TRIALS = 3

# Loop over the date range for `shipdate`
for shipdate in tqdm(list(pd.date_range(min_shipdate, max_shipdate, freq='30D'))):
    # Loop over the range for `discount`
    for discount in tqdm(list(np.arange(min_discount, max_discount + 0.01, 0.01)), leave=False):
        # Loop over the range for `quantity`
        for quantity in tqdm(list(np.arange(min_quantity, max_quantity + 1.00, 1.00)), leave=False):
            params = {'shipdate': shipdate, 'discount': discount, 'quantity': quantity}
            row = dict()
            for trial in range(NUM_TRIALS):
                start = time.time()
                res = con.sql(query_template, params=params)
                elapsed = time.time() - start
                row[f'elapsed_{trial}'] = elapsed
                # If we want to, save the result for a sanity check
                # row[f'res_{trial}'] = res.fetchall()
            table.append({**params, **row})

table = pd.DataFrame(table)
table

  0%|          | 0/85 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,shipdate,discount,quantity,elapsed_0,res_0,elapsed_1,res_1,elapsed_2,res_2
0,1992-01-02,0.00,1.0,0.000790,"[(None,)]",0.000564,"[(None,)]",0.000469,"[(None,)]"
1,1992-01-02,0.00,26.0,0.011708,"[(2924525412.1181,)]",0.004710,"[(2924525412.1181,)]",0.004559,"[(2924525412.1181,)]"
2,1992-01-02,0.05,1.0,0.000427,"[(None,)]",0.000333,"[(None,)]",0.000348,"[(None,)]"
3,1992-01-02,0.05,26.0,0.005367,"[(2392007094.3237,)]",0.005555,"[(2392007094.3237,)]",0.005386,"[(2392007094.3237,)]"
4,1992-01-02,0.10,1.0,0.000459,"[(None,)]",0.000341,"[(None,)]",0.000288,"[(None,)]"
...,...,...,...,...,...,...,...,...,...
505,1998-11-26,0.00,26.0,0.000954,"[(232057.6324,)]",0.001062,"[(232057.6324,)]",0.001079,"[(232057.6324,)]"
506,1998-11-26,0.05,1.0,0.000379,"[(None,)]",0.000301,"[(None,)]",0.000295,"[(None,)]"
507,1998-11-26,0.05,26.0,0.000969,"[(199815.5762,)]",0.001021,"[(199815.5762,)]",0.000982,"[(199815.5762,)]"
508,1998-11-26,0.10,1.0,0.000375,"[(None,)]",0.000351,"[(None,)]",0.000301,"[(None,)]"


In [9]:
table

,shipdate,discount,quantity,elapsed_0,res_0,elapsed_1,res_1,elapsed_2,res_2
0,1992-01-02,0.00,1.0,0.000790,"[(None,)]",0.000564,"[(None,)]",0.000469,"[(None,)]"
1,1992-01-02,0.00,26.0,0.011708,"[(2924525412.1181,)]",0.004710,"[(2924525412.1181,)]",0.004559,"[(2924525412.1181,)]"
2,1992-01-02,0.05,1.0,0.000427,"[(None,)]",0.000333,"[(None,)]",0.000348,"[(None,)]"
3,1992-01-02,0.05,26.0,0.005367,"[(2392007094.3237,)]",0.005555,"[(2392007094.3237,)]",0.005386,"[(2392007094.3237,)]"
4,1992-01-02,0.10,1.0,0.000459,"[(None,)]",0.000341,"[(None,)]",0.000288,"[(None,)]"
...,...,...,...,...,...,...,...,...,...
505,1998-11-26,0.00,26.0,0.000954,"[(232057.6324,)]",0.001062,"[(232057.6324,)]",0.001079,"[(232057.6324,)]"
506,1998-11-26,0.05,1.0,0.000379,"[(None,)]",0.000301,"[(None,)]",0.000295,"[(None,)]"
507,1998-11-26,0.05,26.0,0.000969,"[(199815.5762,)]",0.001021,"[(199815.5762,)]",0.000982,"[(199815.5762,)]"
508,1998-11-26,0.10,1.0,0.000375,"[(None,)]",0.000351,"[(None,)]",0.000301,"[(None,)]"


In [10]:
con.close()